# Konvertierung von XLSX-Daten zu OICP-JSON-Daten für ich-tanke-strom.ch

Hintergrund: Sogenannte "Offline-Provider" können uns keine OICP-Datastreams liefern. Sie erfassen daher ihre statischen Daten anhand einer Excel-Vorlage. Dieses Notebook konvertiert diese Excel-Daten in OICP-konforme Daten und pusht sie an den ich-tanke-strom.ch-Endpoint für die Publikation.

Letzte Aktualisierung des Notebooks: 10.07.2023

2 do:
- Anpassung auf OICP 2.3 (derzeit 2.2)

In [1]:
# Einstellungen
operator = "CH*EWD"
file = "2020.09.09_EWD.xlsx"
upload_pw = "" # PW nach Gebrauch unbedingt wieder entfernen

In [2]:
import numpy as np
import pandas as pd
import json
import requests

file_output = file[:-4] + "json"
ladesaeulen = pd.read_excel(file, 'Ladesäulen')
plugs = pd.read_excel(file, 'Plugs')
authentication = pd.read_excel(file, 'Authentication')
payment = pd.read_excel(file, 'Payment')
valueaddedservices = pd.read_excel(file, 'ValueAddedServices')

def count_records(entity, EvseID):

    count = 0
    
    for index, row in entity.iterrows():
        if EvseID == 0:
            count += 1
        else:
            if row["EvseID"] == EvseID:
                count += 1
    return count

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
# XLSX -> OICP-JSON

counter_ladesaeulen = 0

string = "{\"ActionType\": \"fullLoad\",\"OperatorEvseData\": {\"OperatorID\": \"" + operator + "\",\"EvseDataRecord\":["

#Iter Ladesaeulen
for index, row in ladesaeulen.iterrows():
    counter_plugs = 0
    counter_authentication = 0
    counter_payment = 0
    counter_valueaddedservices = 0
    counter_chargingfacilities = 0

    counter_ladesaeulen += 1
    
    string += "{"
    string += "\"EvseID\": "
    string += "\"" + str(row['EvseID']) + "\","
    string += "\"HotlinePhoneNumber\": "
    string += "\"+" + str(row['HotlinePhoneNumber']) + "\","
    string += "\"IsOpen24Hours\": "
    string += str(row['IsOpen24Hours']).lower() + ","    
    string += "\"IsHubjectCompatible\": "
    string += str(row['IsHubjectCompatible']).lower() + ","  
    string += "\"DynamicInfoAvailable\": "
    string += "\"" + str(row['DynamicInfoAvailable']).lower() + "\"" + ","
    string += "\"Accessibility\": "
    string += "\"" + str(row['Accessibility']) + "\","
    string += "\"Address\": {"
    string += "\"City\": "
    string += "\"" + str(row['Address.City']) + "\","
    string += "\"Country\": "
    string += "\"" + str(row['Address.Country']) + "\","   
    string += "\"Street\": "
    string += "\"" + str(row['Address.Street']) + "\","   
    string += "\"HouseNum\": "
    string += "\"" + str(row['Address.HouseNum']) + "\","   
    string += "\"PostalCode\": "
    string += "\"" + str(row['Address.PostalCode']) + "\""       
    string += "},"
    string += "\"GeoCoordinates\": { \"DecimalDegree\": {"
    string += "\"Latitude\": "
    string += "\"" + str(row['Latitude']) + "\","
    string += "\"Longitude\": "
    string += "\"" + str(row['Longitude']) + "\""     
    string += "}},"    
    
    #PaymentOptions
    string += "\"PaymentOptions\": ["
    for index2, row2 in payment.iterrows():
        if row2["EvseID"] == row["EvseID"]:
            counter_payment += 1
            string += "\"" + str(row2['PaymentOption']) + "\""
            if count_records(payment, row["EvseID"]) > counter_payment:
                string += ","
    string += "],"
    
    #Plugs
    string += "\"Plugs\": ["
    for index3, row3 in plugs.iterrows():
        if row3["EvseID"] == row["EvseID"]:
            counter_plugs += 1
            string += "\"" + str(row3['Plugs']) + "\""
            if count_records(plugs, row["EvseID"]) > counter_plugs:
                string += ","
    string += "],"
    
    #ChargingFacility
    for index6, row6 in plugs.iterrows():
        #Wenn ChargingFacility in Excel leer ist, ergibt das im Pandas DataFrame ein NaN
        if row6["EvseID"] == row["EvseID"] and pd.isna(row6["ChargingFacilities.Power"]) is False:
            counter_chargingfacilities += 1
            if counter_chargingfacilities == 1:
                string += "\"ChargingFacilities\": ["
            string += "{\"Power\": " + str(row6['ChargingFacilities.Power']) + "}"
            if count_records(plugs, row["EvseID"]) > counter_chargingfacilities:
                string += ","
            else:
                string += "],"
    #if counter_chargingfacilities > 1:                
        #string += "],"
    
    #AuthenticationModes
    string += "\"AuthenticationModes\": ["
    for index4, row4 in authentication.iterrows():
        if row4["EvseID"] == row["EvseID"]:
            counter_authentication += 1
            string += "\"" + str(row4['AuthenticationMode']) + "\""
            if count_records(authentication, row["EvseID"]) > counter_authentication:
                string += ","
    string += "]"
    
    #ValueAddedServices
    if count_records(valueaddedservices, row["EvseID"]) > 0:
        string += ", \"ValueAddedServices\": ["
        for index5, row5 in valueaddedservices.iterrows():
            if row5["EvseID"] == row["EvseID"]:
                counter_valueaddedservices += 1
                string += "\"" + str(row5['ValueAddedService']) + "\""
                if count_records(valueaddedservices, row["EvseID"]) > counter_valueaddedservices:
                    string += ","
        string += "]" 
    
    string += "}"
    
    if count_records(ladesaeulen, 0) != counter_ladesaeulen:
        string += ","
    
string += "]}}"

# Write json to file
with open(file_output, "w", encoding='utf8') as write_file:
    write_file.write(string)

In [4]:
# Push new data to ich-tanke-strom

url = 'https://ich-tanke-strom.azurewebsites.net/api/oicp/evsepush/v22/operators/' + operator + '/data-records'
headers = {"Content-Type": "application/json; charset=utf-8"}

# Open file
fo = open(file_output, 'rb')

# Push
response = requests.post(url, headers=headers, data=fo, auth=(operator, upload_pw))

# Close file
fo.close()

print("Status Code", response.status_code)
print("JSON Response ", response.json())

Status Code 403
JSON Response  {'result': False, 'StatusCode': {'Code': 403, 'AdditionalInfo': 'Wrong password.'}}
